In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [4]:
#Variable 
#var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
var ="solidprcptot"
#var ="prcptot"
#var ="dlyfrzthw"
#var ="growing_season_length"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_annual.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+var+"_annual.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_

In [5]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_solidprcptot_annual.nc']

# FOR RCP45 

In [6]:
dsEns45= ens.create_ensemble(files45)
#dsEns45

## Get the percentiles

In [7]:
# Calculate ensembles percentiles on regional means
perctiles45 = ens.ensemble_percentiles(dsEns45)
perctiles45
dsEns45 =0

## Convert in Dataframe

In [9]:
dfper145 = perctiles45.to_dataframe().dropna()
dfper45 = perctiles45.drop('realization').to_dataframe().dropna()
dfmean45 = dfper45.reset_index()
dfper45 = 0

In [10]:
#dfmean45

In [11]:
dfmean45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_forTS.feather")

In [12]:
dfmean45 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_forTS.feather")

# FOR RCP85 

In [13]:
dsEns85= ens.create_ensemble(files85)
#dsEns85

## Get the percentiles

In [14]:
# Calculate ensembles percentiles on regional means
perctiles85 = ens.ensemble_percentiles(dsEns85)
perctiles85
dsEns85 =0

## Convert in Dataframe

In [15]:
dfper185 = perctiles85.to_dataframe().dropna()
dfper85 = perctiles85.drop('realization').to_dataframe().dropna()
dfmean85 = dfper85.reset_index()
dfper85 = 0

In [16]:
#dfmean85

In [17]:
dfmean85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

In [18]:
dfmean85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

# OBSERVED DATA

In [19]:
dsOBs= xr.open_mfdataset(filesobs)

will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  dsOBs= xr.open_mfdataset(filesobs)

to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  combined = auto_combine(



In [20]:
dsOBs

<xarray.Dataset>
Dimensions:       (lat: 510, lon: 1068, time: 64)
Coordinates:
  * time          (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2013-01-01
  * lat           (lat) float32 83.45833 83.375 83.291664 ... 41.125 41.041668
  * lon           (lon) float32 -140.95833 -140.875 ... -52.125004 -52.04167
Data variables:
    solidprcptot  (time, lat, lon) float32 dask.array<chunksize=(64, 510, 1068), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           NRCAN 10km Gridded Climate Dataset
    history:         2012-10-19T16:38:49: Convert from original format to NetCDF
    institution:     NRCAN
    source:          ANUSPLIN
    redistribution:  Redistribution policy unknown. For internal use only.

## Convert in Dataframe

In [21]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0

In [22]:
dfObs = dfmeanObs.rename(columns={var: var+'_Obs'})

In [23]:
dfObs

,lat,lon,time,solidprcptot_Obs
0,83.125000,-74.541672,1950-01-01,97.809998
1,83.125000,-74.541672,1951-01-01,80.720001
2,83.125000,-74.541672,1952-01-01,77.459999
3,83.125000,-74.541672,1953-01-01,126.579994
4,83.125000,-74.541672,1954-01-01,57.630005
...,...,...,...,...
16464827,41.708332,-82.208336,2009-01-01,82.680000
16464828,41.708332,-82.208336,2010-01-01,70.550003
16464829,41.708332,-82.208336,2011-01-01,109.059998
16464830,41.708332,-82.208336,2012-01-01,25.209997


In [24]:
dfObs.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

In [25]:
dfObs = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

## Merge 2 rcp dataframes

In [26]:
df2m =  pd.merge(dfmean45, dfmean85, on=["lat","lon", "time"], suffixes=('_45', '_85'))

In [27]:
df2m

,lat,lon,time,solidprcptot_p10_45,solidprcptot_p50_45,solidprcptot_p90_45,solidprcptot_p10_85,solidprcptot_p50_85,solidprcptot_p90_85
0,66.623306,-89.045212,1950-01-01,124.688156,144.241928,164.693527,125.154518,145.417313,167.477890
1,66.623306,-89.045212,1951-01-01,109.169937,128.686707,149.757523,110.070961,130.774124,149.144318
2,66.623306,-89.045212,1952-01-01,103.477226,133.644638,160.624710,101.982391,134.047134,159.755310
3,66.623306,-89.045212,1953-01-01,102.191559,131.006851,158.441101,102.266777,130.683594,160.392029
4,66.623306,-89.045212,1954-01-01,109.389801,141.920380,169.858826,109.548889,141.883194,169.141571
...,...,...,...,...,...,...,...,...,...
11507101,40.041039,-74.045807,2096-01-01,4.686313,13.790977,59.855244,0.000000,6.462111,37.802505
11507102,40.041039,-74.045807,2097-01-01,6.848152,17.010681,52.099159,2.463510,4.530007,40.702274
11507103,40.041039,-74.045807,2098-01-01,8.020243,32.514015,63.753597,0.000000,5.677419,21.532455
11507104,40.041039,-74.045807,2099-01-01,1.936671,21.796797,29.183802,0.000000,2.411559,8.850348


# Cut by region for rcp45, 85 and Obs

## Get the lat lon regions

In [28]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [29]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

['/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_REG_FOR_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_UA_PER_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/REG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SDOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/Secteurs_Operations_Regionaleslat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SREG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/territoire_guidelat_lon.feather']

In [30]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

## Create DF by region

In [31]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [32]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [33]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

## Cutting rcp45 and 85 by region and getting df with 4th degree polynomial

In [34]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [35]:
dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 


NameError: name 'dfdataregObs' is not defined

In [ ]:
dfmeanObs

In [ ]:
listdf2m = []

for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/8... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/8... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, df2m, on=["lat","lon"])
    dfdataregObs = pd.merge(dfregion, dfObs, on=["lat","lon"])
    print ("3/8... Get mean by year for every sub-region")
    dfmeanUA45 = dfdatareg45
    dfmean45UA = dfmeanUA45.groupby(["time", name]).mean()
    dfmean45UA2 = dfmean45UA.reset_index()
    dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 
    dfmeanObs2 = dfmeanObs.reset_index()
    print("Merging 3 cases")
    df3m = pd.merge(dfmean45UA2, dfmeanObs2, how='left', on= ["time", name] )
    print ("4/8... Split by subregion")
    df3ma = df3m[['time', name, var+'_p10_45', var+'_p50_45', var+'_p90_45',
                               var+'_p10_85', var+'_p50_85', var+'_p90_85', var+'_Obs']]
    if (var == "tg_mean" or var== "tn_mean" or var==):
        df3ma[var+"_Obs"] = df3ma[var+"_Obs"]- 273.15
    if (var == "solidprcptot"):
        df3ma[var+"_Obs"] = df3ma[var+"_Obs"]/10
    df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)
    for name2,dfsr in df3ma.groupby(name):
        print ("                   "+name2, name)
        dfsrc = dfsr.copy()
        print ("6/8... 4th Polynomial - rcp45,85")
        years = dfsrc.time.dt.year
        dfsrc["time"] = years#.values
        for p in ('_p10_45', '_p50_45', '_p90_45', '_p10_85', '_p50_85', '_p90_85'):
            if (var == "tg_mean" or var== "tn_mean" or var==):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4) - 273.15
            if (var == "solidprcptot"):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4)/10
            dfsrc[var+p] = round(dfsrc[var+p], 2) 
        print(" -----------------------8/8 Creating CSV file for sub-region " +name2)
        dfsrc.to_csv(replace_all(name2,d)+"_"+var+"_annual.csv")
    
    
                            
       